In [1]:
import glob
import os
import pandas as pd
from tqdm import tqdm
import numpy as np
import pysam
import time
from collections import defaultdict
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import subprocess
%matplotlib inline

from os.path import join

In [2]:
FUNC_PATH = "/home/isshamie/software/mito-genotyping/exampleProcessing/"
BAM = "data/aml035_post_transplant_possorted_genome_bam.MT.bam"
BARCODES = "../data/raw/Human2_002_Barcodes.txt"
BARCODE_INFO = "data/barcode_data_aml035_post_transplant_possorted_genome_bam.MT.p" #"data/scBam/barcode_data.p"
NUM_CORES = 32
IN = "data/scBam"
OUT_DIR = "data/scPileup/"
if not os.path.exists(OUT_DIR):
    os.mkdir(OUT_DIR)

In [3]:
files = glob.glob(IN+"/*")
len(files)

44018

In [4]:
over_200 = set()
count = 0
for i in tqdm(files):
    output = subprocess.check_output(f"samtools view {i} | wc -l", shell=True, universal_newlines=True )
    if int(output.strip()) >= 200:
        over_200.add(i)
        count += 1
        if count > 100:
            break
        
over_200 = list(over_200)

  7%|▋         | 2890/44018 [00:57<13:57, 49.09it/s]

In [5]:
count

101

## Load CB's, get ones that have over 200 reads and put in separate folder

In [6]:
[CR_read_number,CB_read_number,BC_read_number, barcodes, corrected_barcodes, barcode_pairs] = pickle.load(open(BARCODE_INFO,"rb"))

In [7]:
CB_read_200 = set()
for i in CB_read_number:
    if CB_read_number[i] >= 200:
        CB_read_200.add(i)
print(len(CB_read_200))
CB_read_200 = np.array(list(CB_read_200))

1426


## Calculate coverage of each position 

In [9]:
bamfile = list(over_200)[0]
outpre = join(OUT_DIR, os.path.basename(bamfile.replace(".bam","")))
maxBP = 16571
base_qual = 0 
sample = os.path.basename(bamfile).split("_")[-1]
alignment_quality = 0

# bamfile = sys.argv[1] # Filepath to raw bamfile
# outpre = sys.argv[2] # Prefix / basename for raw file output
# maxBP = sys.argv[3] # Maximum length of mtDNA genome
# base_qual = float(sys.argv[4]) # Minimum base quality to be considered for pileup
# sample = sys.argv[5] # Sample name to be considered for downstream analyses
# alignment_quality = float(sys.argv[6]) # minimum alignment quality required to be considered


In [10]:
cmd = f"python {FUNC_PATH}/01_pileup_counts.py {bamfile} {outpre} {maxBP} {base_qual} {sample} {alignment_quality}"
print(cmd)
os.system(cmd)

python /home/isshamie/software/mito-genotyping/exampleProcessing//01_pileup_counts.py data/scBam/scBamCB_13967.bam data/scPileup/scBamCB_13967 16571 0 13967.bam 0


0